1. 데이터셋 추가 및 통합 📚

- KoAlpaca 데이터셋 로드 : 먼저 datasets 라이브러리를 사용해 공개된 한국어 데이터셋인 beomi/KoAlpaca-v1.1a를 불러옵니다.
- 데이터 형식 맞추기 : KoAlpaca 데이터셋의 컬럼 이름('instruction', 'output')을 기존 KoChatGPT 데이터셋과 동일하게('prompt', 'completion') 변경합니다.
- 데이터셋 통합 : 기존에 사용하던 kochatgpt_1_SFT.jsonl 데이터와 새로 불러온 KoAlpaca 데이터를 하나로 합쳐 combined_data라는 통합 데이터셋을 만듭니다. 이를 통해 모델이 더 다양하고 많은 양의 데이터를 학습할 수 있게 됩니다.

In [ ]:
from datasets import load_dataset, concatenate_datasets

# 1. Ko-Alpaca 데이터셋 로드
new_data = load_dataset("beomi/KoAlpaca-v1.1a", split="train")

# 2. 데이터 형식 변환 함수
# Ko-Alpaca는 'instruction'과 'output'으로 구성되어 있으므로,
# 이를 'prompt'와 'completion'으로 이름을 바꿔줍니다.
def transform_ko_alpaca(example):
    return {
        'prompt': example['instruction'],
        'completion': example['output']
    }

# 함수를 적용하여 컬럼 이름 변경
new_data_transformed = new_data.map(transform_ko_alpaca, remove_columns=new_data.column_names)

# 3. 기존 데이터셋 로드 (만약 로컬에 저장된 jsonl을 사용한다면)
# 이전에 data 변수에 로드했다면 그 변수를 그대로 사용해도 됩니다.
# 여기서는 다시 로드하는 코드를 예시로 보여드립니다.
original_data = load_dataset("json", data_files="KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl", split="train")


# 4. 두 데이터셋 통합
# concatenate_datasets 함수를 사용하여 두 데이터를 합칩니다.
combined_data = concatenate_datasets([original_data, new_data_transformed])

print(f"--- 데이터셋 통합 완료 ---")
print(f"원본 데이터셋 크기: {len(original_data)}")
print(f"추가된 데이터셋 크기: {len(new_data_transformed)}")
print(f"통합 후 전체 데이터셋 크기: {len(combined_data)}")

# 메모리 관리를 위해 사용하지 않는 변수는 삭제
del new_data, new_data_transformed, original_data

2. 데이터 정제
- 필터링 : 통합된 데이터셋(33,155개)에서 'prompt'나 'completion'의 길이가 5글자 이하인 짧은 데이터들을 제거하여, 최종적으로 32,906개의 데이터를 학습에 사용하도록 정제합니다.

In [4]:
# 데이터셋을 정제하는 함수입니다.
def refine_dataset(example):
    if example['prompt'] is None or example['completion'] is None:
        return False
    return len(example['prompt']) > 5 and len(example['completion']) > 5

# 'combined_data'를 입력으로 사용합니다.
refined_data = combined_data.filter(refine_dataset, num_proc=4)

print(f"정제 전 데이터셋 크기: {len(combined_data)}")
print(f"정제 후 데이터셋 크기: {len(refined_data)}")

Filter (num_proc=4):   0%|          | 0/33155 [00:00<?, ? examples/s]

정제 전 데이터셋 크기: 33155
정제 후 데이터셋 크기: 32906


3. 모델 학습 준비 및 실행 ⚙️

- 모델 및 토크나이저 로드 : 첫 번째 실험과 동일하게 skt/kogpt2-base-v2 모델과 토크나이저를 준비합니다.
- 데이터 전처리 : 통합되고 정제된 텍스트 데이터를 모델이 이해할 수 있는 숫자 형태(토큰)로 변환합니다. 이때 모델이 'completion'(응답) 부분만 학습하도록 'prompt'(지시) 부분은 레이블에서 제외(-100으로 마스킹)합니다.
- 모델 학습 : Transformers 라이브러리의 Trainer를 사용해 추가된 데이터로 미세 조정(SFT) 학습을 진행하도록 설정합니다.

In [ ]:
# KoGPT2 모델과 토크나이저를 로드합니다.
model_name = "skt/kogpt2-base-v2"
base_model = AutoModelForCausalLM.from_pretrained(model_name)
base_tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', pad_token='<pad>')

In [ ]:
# 데이터를 모델 입력 형식에 맞게 토크나이징하는 함수입니다.
def preprocess(example):
    # BOS(Beginning of Sentence)와 EOS(End of Sentence) 토큰을 추가합니다.
    prompt = base_tokenizer.bos_token + example['prompt']
    completion = example['completion'] + base_tokenizer.eos_token
    
    # 프롬프트와 정답을 합쳐 전체 텍스트를 생성합니다.
    full_text = prompt + completion
    
    # 토크나이징을 수행합니다.
    encodings = base_tokenizer(full_text, truncation=True, padding='max_length', max_length=256)
    
    # 레이블을 생성합니다. 프롬프트 부분은 loss 계산에서 제외하기 위해 -100으로 마스킹합니다.
    labels = encodings['input_ids'][:]
    prompt_tokens = len(base_tokenizer(prompt, truncation=True)['input_ids'])
    labels[:prompt_tokens] = [-100] * prompt_tokens
    
    encodings['labels'] = labels
    return encodings

# 정제된 데이터셋에 전처리 함수를 적용합니다.
processed_data = refined_data.map(preprocess, remove_columns=refined_data.column_names)

In [ ]:
# 학습을 위한 인자들을 설정합니다.
training_args = TrainingArguments(
    output_dir="./sft-output",
    num_train_epochs=3, # 시간 관계상 1 에폭만 설정, 실제로는 더 많은 학습 필요
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
)

# SFT를 수행할 모델과 토크나이저를 복사하여 사용합니다.
sft_model = AutoModelForCausalLM.from_pretrained(model_name)
sft_tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', pad_token='<pad>')

# Trainer 객체를 생성합니다.
trainer = Trainer(
    model=sft_model,
    args=training_args,
    train_dataset=processed_data,
)

# 학습을 시작합니다.
trainer.train()

4. 모델 성능 평가 및 비교 🔍

- 정성적 평가 (사용자 입력 기반) : 
    - 사용자가 직접 프롬프트를 입력하면, 원본 KoGPT-2 모델과 새롭게 미세 조정한 SFT 모델이 각각 생성한 답변을 나란히 보여줍니다.
    - 특히 SFT 모델은 Greedy, Beam Search, Top-k Sampling 등 다양한 생성 방식으로 답변을 만들어 성능을 다각도로 비교할 수 있게 합니다.

- 정량적 평가 (ROUGE 점수) :
    - 모델이 생성한 답변과 실제 정답이 얼마나 유사한지를 ROUGE 점수라는 지표로 측정합니다.
    - ROUGE-1, ROUGE-2, ROUGE-L 점수를 통해 단어, 어구, 문장 구조의 유사도를 각각 수치로 평가하여 모델 성능을 객관적으로 확인합니다.

In [ ]:
# 답변을 생성하는 함수
def generate_answer(model, tokenizer, prompt, generation_type, max_length=128):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    
    output_sequences = None
    if generation_type == "greedy":
        output_sequences = model.generate(input_ids, max_length=max_length, do_sample=False)
    elif generation_type == "beam_search":
        output_sequences = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
    elif generation_type == "top_k_sampling":
        output_sequences = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50)

    return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

# 모델들을 평가 모드로 설정
base_model.eval()
sft_model.eval()

# GPU가 사용 가능하다면 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)
sft_model.to(device)


# 사용자 입력을 받아 답변 생성 및 비교
while True:
    prompt = input("프롬프트를 입력하세요 (종료하려면 'exit' 입력): ")
    if prompt.lower() == 'exit':
        break
    
    print("--------------------------------------------------")
    print(f"[입력 프롬프트]: {prompt}")
    print("--------------------------------------------------")
    
    # 원본 KoGPT-2 답변
    base_answer = generate_answer(base_model, base_tokenizer, prompt, "greedy")
    print(f"[KoGPT-2 원본]: {base_answer}")
    print("---")
    
    # SFT 모델 답변
    sft_greedy = generate_answer(sft_model, sft_tokenizer, prompt, "greedy")
    sft_beam = generate_answer(sft_model, sft_tokenizer, prompt, "beam_search")
    sft_top_k = generate_answer(sft_model, sft_tokenizer, prompt, "top_k_sampling")
    print(f"[SFT - Greedy]: {sft_greedy}")
    print(f"[SFT - Beam Search]: {sft_beam}")
    print(f"[SFT - Top-k Sampling]: {sft_top_k}")
    print("--------------------------------------------------\n")

In [ ]:
# ROUGE 평가 지표를 로드합니다.
rouge = evaluate.load('rouge')

# 평가를 위한 데이터셋 일부를 샘플링합니다. (시간 단축을 위해 100개만 사용)
eval_dataset = refined_data.shuffle(seed=42).select(range(100))

# SFT 모델의 예측과 실제 정답을 저장할 리스트
predictions = []
references = []

for item in eval_dataset:
    prompt = item['prompt']
    reference = item['completion']
    
    # SFT 모델로 예측 생성
    prediction = generate_answer(sft_model, sft_tokenizer, prompt, "greedy")
    # 생성된 답변에서 프롬프트를 제거합니다 (순수 생성 부분만 남기기 위함).
    if prediction.startswith(prompt):
        prediction = prediction[len(prompt):]
        
    predictions.append(prediction)
    references.append(reference)
    
# ROUGE 점수 계산
# nltk.sent_tokenize를 사용하여 문장 단위로 분리 후 계산합니다.
rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: nltk.sent_tokenize(x))

# 결과 출력
print("--- 정량적 평가 (ROUGE Score) ---")
print(f"ROUGE-1: {rouge_results['rouge1'] * 100:.2f}") # Unigrams (단일 단어) 기반 유사도
print(f"ROUGE-2: {rouge_results['rouge2'] * 100:.2f}") # Bigrams (연속된 두 단어) 기반 유사도
print(f"ROUGE-L: {rouge_results['rougeL'] * 100:.2f}") # Longest Common Subsequence (가장 긴 공통 부분 문자열) 기반 유사도